# Save Markdown text into Vector DB

## Step-1: Config

In [1]:
from my_config import MY_CONFIG

## Step-2: Read Markdown

In [2]:
import os
import glob

pattern = os.path.join(MY_CONFIG.OUTPUT_DIR, '*.md')
md_file_count = len(glob.glob(pattern, recursive=True)) 

In [3]:
from llama_index.core import SimpleDirectoryReader

reader = SimpleDirectoryReader(input_dir=MY_CONFIG.OUTPUT_DIR, recursive=True )
documents = reader.load_data()

print (f"Loaded {len(documents)} documents from {md_file_count} files")


Loaded 87 documents from 87 files


In [4]:
## Inspect a sample doc
print (documents[0])

Doc ID: add0569b-f827-49cf-9972-ade596c3e5cc
Text: # Building the open future of AI  We are technology developers,
researchers, industry leaders and advocates who collaborate to advance
safe, responsible AI rooted in open innovation.  <!-- image -->  ###
Skills &amp; Education  <!-- image -->  Supporting global AI skill-
building, education, and exploratory research.  ### Trust &amp; Safety
<!--...


## Step-3: Create Chunks

In [5]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(chunk_size=MY_CONFIG.CHUNK_SIZE, chunk_overlap=MY_CONFIG.CHUNK_OVERLAP)
nodes = parser.get_nodes_from_documents(documents)
print(f"Created {len(nodes)} chunks from {len(documents)} documents")

Created 202 chunks from 87 documents


## Step-4: Setup Embedding Model

In [6]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [7]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)

/home/sujee/apps/anaconda3/envs/rag-website-3-docling/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step-5: Connect to Milvus

In [8]:
## Clear up any old data

from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)
print ("✅ Connected to Milvus instance: ", MY_CONFIG.DB_URI )

# if we already have a collection, clear it first
if milvus_client.has_collection(collection_name = MY_CONFIG.COLLECTION_NAME):
    milvus_client.drop_collection(collection_name = MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)
    

✅ Connected to Milvus instance:  ./rag_website.db
✅ Cleared collection : pages


In [9]:
# connect llama-index to vector db

from llama_index.core import StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected Llama-index to Milvus instance: ", MY_CONFIG.DB_URI )

2025-03-12 13:19:00,906 [DEBUG][_create_connection]: Created new connection using: 5cb1a639a33a4bc9b505140625a3c3ac (async_milvus_client.py:600)


✅ Connected Llama-index to Milvus instance:  ./rag_website.db


## Step-6: Save to DB

In [10]:
%%time

## We save entire md documents into vector store

# from llama_index.core import VectorStoreIndex

# index = VectorStoreIndex.from_documents(
#     documents, storage_context=storage_context
# )
# print (f"✅ Saved {len(documents)} documents to db: {MY_CONFIG.DB_URI}" )

CPU times: user 15 μs, sys: 4 μs, total: 19 μs
Wall time: 38.4 μs


In [11]:
%%time 

# save chunks into vector db

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex(
        nodes=nodes,
        storage_context=storage_context,
    )

print(f"Successfully stored {len(nodes)} chunks in Milvus collection '{MY_CONFIG.COLLECTION_NAME}'")


Successfully stored 202 chunks in Milvus collection 'pages'
CPU times: user 810 ms, sys: 125 ms, total: 935 ms
Wall time: 690 ms
